## How to map a star onto a CCD

In [ ]:
import requests

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from astroquery.mast import Observations
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table

%matplotlib inline

### Invoke your target

In [ ]:
objname = "LHS 3844" 

### request info from exo.MAST

In [ ]:
planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
header={}
url = planeturl + "/identifiers/"

myparams = {"name":objname}
r = requests.get(url = url, params = myparams, headers = header)

planet_names = r.json() 
print(planet_names)

In [ ]:
ticid = planet_names['tessID']
tce = planet_names['tessTCE']
Ra, Dec = planet_names['ra'],planet_names['dec']

### Pull a single calibrated FFI for the target

In [ ]:
from astroquery.mast import Observations
obsTable = Observations.query_object(objname,radius=".02 deg")
dataProductsByObservation = Observations.get_product_list(obsTable[0:2])  # grab just the TESS ones

In [ ]:
# obsTable.to_pandas() # table of observations from different observers (TESS, GALEX, etc.)

In [ ]:
ffics = Observations.filter_products(dataProductsByObservation,productSubGroupDescription='FFIC')

In [ ]:
single_ffic = Table(ffics[100])

### Import FFI 

In [ ]:
manifest = Observations.download_products(single_ffic,'../data')

In [ ]:
url = manifest['Local Path'][0]

In [ ]:
fits.info(url)

### Plot FFI

In [ ]:
with fits.open(url, mode = "readonly") as hdulist:
    wcs_info = WCS(hdulist[1].header)
    cal_image = hdulist[1].data
    header = hdulist[1].header

In [ ]:
cam, ccd = header['CAMERA'], header['CCD']
cam, ccd

In [ ]:
sector = url.split("-")[1]

In [ ]:
# Use the header to determine the mid-point of the exposure time for this FFI.
mid_time = np.around([(header['TSTOP'] + header['TSTART']) / 2], decimals=3)[0]
mid_time

In [ ]:
# From MAST beginner_how_to_use_ffi.ipynb tutorial: 

# Plot gridded FFI

plt.figure(figsize = (12,12))
ax = plt.subplot(111, projection = wcs_info)

ax.imshow(cal_image, cmap = plt.cm.bone, vmin = np.percentile(cal_image,4),vmax = np.percentile(cal_image, 98), origin = "lower")
plt.grid(axis = 'both', color = 'white', ls = 'solid')

plt.title("TESS Calibrated FFI for Sector {}, Camera {}, CCD {}, Timestamp {} BTJD".format(sector[-1], cam, ccd,mid_time), pad=27)
# plt.xlabel('RA', labelpad=0.3)
# plt.ylabel('Dec',labelpad=-30)

r = ax.coords['ra']
d = ax.coords['dec']

r.set_axislabel('RA')
d.set_axislabel('Dec')

#r.set_major_formatter('hh:mm:ss')
r.set_major_formatter('d.ddd')
d.set_major_formatter('d.ddd')

#r.set_ticks(spacing=180. * u.arcmin)
r.set_ticks(number=20.)
#d.set_ticks(spacing=0.5 * u.deg)
d.set_ticks(number=15.)

r.set_ticks_position('bt')
r.set_ticklabel_position('bt')

d.set_ticks_position('lr')
d.set_ticklabel_position('lr')

starLoc = wcs_info.all_world2pix([[Ra,Dec]],0)  #Second is origin
plt.scatter(starLoc[0,0], starLoc[0,1], s = 150, facecolors='none', edgecolors='r')

# plt.savefig('{}-ccd-near-tois-marked-labeled.png'.format(objname))


### Import TOI List

Add in list of TOIs and filter by RA, Dec.

This list is from ExoFOP, filtered to only include targets which are not 'FP' or false positives.

You can download the full list here https://exofop.ipac.caltech.edu/tess/view_toi.php

In [ ]:
toilist = "exofop_tois_s01_s09_sans_fps.csv"

table = pd.read_csv(toilist)

### Cut TOI list down to just nearby stars

We want RAs between x and y. Plot the FFI with WCS and then play with the ranges for upper and lower limits for RA and Dec.

In [ ]:
index1 = (table['Dec (degrees)'] > -81.)*(table['Dec (degrees)'] < -67.)
# cut1 = table_all[['RA (degrees)','Dec (degrees)']][index1]
cut1 = table[index1]
index2 = (cut1['RA (degrees)'] > 325.)|(cut1['RA (degrees)'] < 15.)
nearby = cut1[['TIC ID','TOI','RA (degrees)','Dec (degrees)']][index2]

In [ ]:
nearby_tois = nearby[['RA (degrees)','Dec (degrees)']].get_values()
nearby_names = nearby['TOI'].get_values()

### Plot again now with nearby TOIs

In [ ]:
# From MAST beginner_how_to_use_ffi.ipynb tutorial: 

# Plot gridded FFI

plt.figure(figsize = (12,12))
ax = plt.subplot(111, projection = wcs_info)

ax.imshow(cal_image, cmap = plt.cm.bone, vmin = np.percentile(cal_image,4),vmax = np.percentile(cal_image, 98), origin = "lower")
plt.grid(axis = 'both', color = 'white', ls = 'solid')

plt.title("TESS Calibrated FFI for Sector {}, Camera {}, CCD {}, Timestamp {} BTJD".format(sector[-1], cam, ccd,mid_time), pad=27)
# plt.xlabel('RA', labelpad=0.3)
# plt.ylabel('Dec',labelpad=-30)

r = ax.coords['ra']
d = ax.coords['dec']

r.set_axislabel('RA')
d.set_axislabel('Dec')

#r.set_major_formatter('hh:mm:ss')
r.set_major_formatter('d.ddd')
d.set_major_formatter('d.ddd')

#r.set_ticks(spacing=180. * u.arcmin)
r.set_ticks(number=20.)
#d.set_ticks(spacing=0.5 * u.deg)
d.set_ticks(number=15.)

r.set_ticks_position('bt')
r.set_ticklabel_position('bt')

d.set_ticks_position('lr')
d.set_ticklabel_position('lr')

starLoc = wcs_info.all_world2pix([[Ra,Dec]],0)  #Second is origin
plt.scatter(starLoc[0,0], starLoc[0,1], s = 150, facecolors='none', edgecolors='r')

# plt.savefig('ffis/{}-ccd-near-tois-marked-labeled.png'.format(objname))


nearbyLoc = wcs_info.all_world2pix(nearby_tois[:], 0)
plt.scatter(nearbyLoc[:,0], nearbyLoc[:,1], s = 25, color = 'orange')

# names = ["WASP-91 b","TOI 135.01","LHS 3844 b","TOI 200.01"]
names = nearby_names
for i, txt in enumerate(names):
    ax.annotate(txt,
                xy=(nearbyLoc[i,0]+10,nearbyLoc[i,1]+10),
                color='white',
                weight='bold',
                size='medium')

In [ ]:
# plt.savefig('{}-ccd-near-tois-marked-labeled.png'.format(objname))